In [1]:
"""import os
import pandas as pd

directory = r'D:\Audios_fm'

wav_files = [f for f in os.listdir(directory) if f.endswith('.wav')]
df = pd.DataFrame(wav_files, columns=['audio_name'])

output_file ='signal detection.xlsx'

df.to_excel(output_file, index=False)

print(f'list of audio files saved to {output_file}')"""

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\david\AppData\Local\Temp\ipykernel_6216\1645478713.py:1: SyntaxWarning: invalid escape sequence '\A'
  """import os


"import os\nimport pandas as pd\n\ndirectory = r'D:\\Audios_fm'\n\nwav_files = [f for f in os.listdir(directory) if f.endswith('.wav')]\ndf = pd.DataFrame(wav_files, columns=['audio_name'])\n\noutput_file ='signal detection.xlsx'\n\ndf.to_excel(output_file, index=False)\n\nprint(f'list of audio files saved to {output_file}')"